In [1]:
import os
import numpy as np
import pandas as pd
from bids import BIDSLayout
from nilearn import datasets
from nilearn import input_data
from nilearn.interfaces.fmriprep import load_confounds

In [7]:
def process_data_bids(bids_root, strategy, atlas_name):
    print("BIDS root:", bids_root)
    print("Directories and files at BIDS root:", os.listdir(bids_root))

    atlases = {
        'schaefer400': datasets.fetch_atlas_schaefer_2018(n_rois=400),
        'schaefer1000': datasets.fetch_atlas_schaefer_2018(n_rois=1000),
        "yeo": datasets.fetch_atlas_yeo_2011()
    }
    if atlas_name not in atlases:
        raise ValueError(f"Atlas not supported. Available options: {', '.join(atlases.keys())}")

    atlas_data = atlases[atlas_name]
    if atlas_name == 'yeo':
        atlas_filename = atlas_data['thin_17']
    else:
        atlas_filename = atlas_data.maps
    
    layout = BIDSLayout(bids_root, validate=False, derivatives=True, absolute_paths=True)

    masker = input_data.NiftiLabelsMasker(labels_img=atlas_filename, standardize=True, verbose=2)

    all_data = []
    subjects = layout.get_subjects()
    for subject_id in subjects:
        sessions = layout.get_sessions(subject=subject_id)
        for session in sessions:
            func_files = layout.get(subject=subject_id, session=session, suffix='bold', extension='nii.gz', return_type='filename')
            func_files = [func_files[0]] # only take the first file for debugging/code creation purposes # comment out in actual analysis
            for func_file in func_files:
                task = layout.get_metadata(func_file).get('TaskName', 'unknown')
                
                if strategy == 'gsr':
                    confounds = load_confounds(func_file, strategy=['global_signal', "high_pass", "wm_csf"])
                elif strategy == 'compcor':
                    confounds = load_confounds(func_file, strategy=['motion', "high_pass", "wm_csf"], motion="basic", compcor='anat_combined', n_compcor='all')
                else:
                    raise ValueError(f"Unknown strategy: {strategy}. Available options: gsr, compcor")
                
                time_series = masker.fit_transform(func_file, confounds=confounds[0])
                df = pd.DataFrame(time_series, columns=[f'Region_{i}' for i in range(time_series.shape[1])])
                df['Subject'] = subject_id
                df['Session'] = session
                df['Task'] = task

                all_data.append(df)
                break # stop after processing first file of first subject # comment out in full analysis

        break # stop after processing first session of first subject # comment out in full analysis    

    return pd.concat(all_data, ignore_index=True) if all_data else pd.DataFrame()

In [8]:
# path parsa
# bids_root = r"C:\Users\prsyu\OneDrive\Bidlung\University\M.S. Leiden University\M.S. Neuroscience (Research)\Thesis\CopBET\CopBET_Python\nilearn_data\development_fmri"

# path olivier
#bids_root = "/Users/olivier/nilearn_data/development_fmri"
bids_root = r"/home/s1836706/data_pi-michielvanelkm/ascdb/basel_LAM"

strategy = 'compcor'
atlas_name = "yeo"

results = process_data_bids(bids_root, strategy, atlas_name)
print(results)

results_firstsubj = process_data_bids(bids_root, strategy, atlas_name)
print(results_firstsubj)


save_path = r"/home/s1836706/data_pi-michielvanelkm/Olivier/csv_attempt_1"
results.to_csv(save_path, index=False)



BIDS root: /Users/olivier/nilearn_data/development_fmri
Directories and files at BIDS root: ['.DS_Store', 'README.md', 'development_fmri']
Processing subject: pixar001
Sessions for subject pixar001: []
Processing subject: pixar002
Sessions for subject pixar002: []
Processing subject: pixar003
Sessions for subject pixar003: []
Processing subject: pixar004
Sessions for subject pixar004: []
Processing subject: pixar005
Sessions for subject pixar005: []
Processing subject: pixar006
Sessions for subject pixar006: []
Processing subject: pixar007
Sessions for subject pixar007: []
Processing subject: pixar008
Sessions for subject pixar008: []
Processing subject: pixar123
Sessions for subject pixar123: []
Processing subject: pixar124
Sessions for subject pixar124: []
Empty DataFrame
Columns: []
Index: []


In [ ]:

def process_and_save_to_matlab(df, eng):
    if df.empty:
        print("No data to process.")
        return None
    try:
        # Assuming the last columns are 'Subject', 'Session', 'Task', 'Dataset'
        mat_data = matlab.double(df.iloc[:, :-4].values.tolist())
        subjects = df['Subject'].tolist()
        sessions = df['Session'].tolist()
        tasks = df['Task'].tolist()
        datasets = df['Dataset'].tolist()
        num_vols = [len(df)] * len(df)

        mat_struct = eng.struct('data', mat_data, 'subject', subjects, 'session', sessions, 'task', tasks, 'dataset', datasets, 'num_vols', num_vols)
        mat_table = eng.struct2table(mat_struct, 'AsArray', True)
        return mat_table
    except Exception as e:
        print(f"Failed to create MATLAB table: {str(e)}")

results = process_data_bids(bids_root, strategy, atlas_name)

matlab_tbl = process_and_save_to_matlab(results, eng)

if matlab_tbl:
    eng.workspace['matlab_tbl'] = matlab_tbl  # Assign table to MATLAB workspace
    eng.save(save_path, 'matlab_tbl')  # This should save the table directly without returning it
    print("MATLAB table saved.")
else:
    print("No table to save.")
eng.quit()

eng = matlab.engine.start_matlab()
